In [5]:
import numpy as np

In [264]:
class BinaryTree():
    '''
    Train Data:
    X in shape (m, n)
    m = num_samples, n = num_features,
    Only one_hot values, no multiclass or regression
    Features Must Not be single valued: e.g 
    >>> X = np.array([[0],[0],[0],[0],[0],[0],[0]])
    >>> y = np.array([ 0 , 1 , 0 , 0 , 1 , 1 , 0 ])
    '''
    class Node():

        def __init__(self, depth=0, parent=None):
            self.depth = depth
            self.parent = parent
            self.h = 1

    class Decisive(Node):

        def __init__(self, feature, depth=0, parent=None):
            self.feature = feature
            self.children = {}
            super().__init__(depth=depth, parent=parent)
            


    class Root(Decisive):
        
        def __init__(self,feature):
            super().__init__(feature=feature, )

    class Decision(Decisive):
        
        def __init__(self, feature, parent, depth=0, ):
            super().__init__(feature=feature, depth=depth,  parent=parent)

    class Leaf(Node):
        
        def __init__(self, parent, confidence=1, depth=1,output=1):
            self.confidence = confidence
            self.output = output
            super().__init__(depth=depth, parent=parent)
            

    def __init__(self, threshold=1, depth=3):
        self.threshold = np.clip(threshold, 0, 1)
        self.max_depth=np.maximum(1,depth)
        self.root  = None
        self.current_feature = None
       


    def H(self, x):
        if x == 0 or x ==1:
            return 0
        return -x*np.log2(x) - (1-x) * np.log2(1-x)
    
    def split(self, X, feature):
        '''
        Returns
        ======================
        feature=True, feature=False
        '''
        return X[X[:,feature]==1], X[X[:,feature]==0]
        
    def information_gain(self, X, y, feature):
        '''
        Return 
        =================
        Information gain, whether to flip positive and negative
        '''
        if y.size == 0:
            raise KeyError('Y got size of zero')
        if y.mean() == 0 or y.mean() == 1:
            raise ValueError('Completely Pure Node recieved')
        X = np.concatenate([X,y.reshape(-1,1)], axis=-1)

        tot = X.shape[0]
        
        positive, negative = self.split(X,feature)
        
        if positive.size == 0 or negative.size== 0:
            return 0

        w1, w0 = positive.shape[0]/tot ,negative.shape[0]/tot
        
        p1, p0 =  (positive[:,-1]).mean(), (negative[:,-1]).mean()

        return self.H(np.mean(y)) - (w1 * self.H(p1) + w0 * self.H(p0))

    def get_best(self, X :np.ndarray,y :np.ndarray) -> int:
        '''
        Get the best feature to split on
        
        Returns :
        _________________________________
        
        best_feature_column
        '''
        best = 0
        max = 0
        for i in range(X.shape[-1]):
            x= self.information_gain(X,y,i)
            if best < x:
                best = x
                max = i
        return max


    def train(self, X, y, node : Node= None):
        
        end = False
        # print('Current data\n', X, '\n', y)
        removes = []
        for f in range(X.shape[-1]):
            if X[:,f].mean()==0 or X[:,f].mean()==1:
                removes.append(f)
        if len(removes):
            X=np.delete(X, removes, axis=-1)
            # print('-----------------------------------')
            # print('Current data\n', X, '\n', y)
        if node is not None:
            prob = y.mean()
            p = np.maximum(prob, 1-prob)
            print('Confidence', p)
            node.h = self.H(p)
            if p >= self.threshold:
                end = True
                print('Threshold Met ')
            elif X.size == 0:
                end=True
                print('End of Features')
            elif node.depth == self.max_depth:
                end=True
                print('Maximum Depth Reached')
            if end:
                node = self.Leaf(parent=node.parent, confidence=p, depth=node.depth)
                node.output = int(prob>=0.5)
                node.h = self.H(p)
                return node
            print('Threshold Not met ')

        if node is None:
            print('Initializing Root')
            node = self.Root(feature=None)
            self.root =  node
            node.h = self.H(y.mean())
            self.X = X
            self.y = y
        
        self.current_feature = node
        

        print('Getting Best feature')
        best_feature = self.get_best(X, y)
        self.current_feature.feature = best_feature
        print('Retrieved best feature : ',best_feature)

        p,n = self.split(X = np.concatenate([X,y.reshape(-1,1)], axis=-1), feature=best_feature)
        positive,  pos_y, negative, neg_y = p[:,:-1], p[:,-1], n[:,:-1], n[:,-1]
        # print('Positive split : \n', positive,'\n', pos_y)
        # print('Negative split : \n', negative,'\n', neg_y)
   
        for i, (X,y) in enumerate(([positive, pos_y], [negative, neg_y])):
            print(f'Setting Child {i} of Depth' ,node.depth+1)
            node.children[i] = self.train(X=np.delete(X, best_feature, axis=-1), y=y,node=self.Decisive(feature=None,depth=node.depth+1, parent=node))
        return node
    
    def pretty_print(self, node=None):

        
        if node is None:
            node = self.root
        print('='+'======'*node.depth+'>', 'Depth :', node.depth,'    |||     Entropy : ', node.h, end='')
        if type(node) == self.Leaf:
            print('   |||    Confidence', node.confidence)
            return
        else:
            print('   |||    Feature', node.feature)
        for child in node.children.values():
            self.pretty_print(child)
        
    def predict(self, X, node=None):

        if node is None:
            node = self.root
        elif type(node) == self.Leaf:
            return np.random.choice([node.output, 1 - node.output], p=[node.confidence, 1- node.confidence])
        for child in node.children.values():
            self.pretty_print(child)
         


In [202]:
features = np.random.randint(size =(10,3), low=0, high=2)
labels = np.array([1,1,1,1,1,0,0,0,0,0])
features = np.array([
    [0,0,0,0,0,0,1,1,1,1],
    [0,0,1,0,0,1,1,1,1,0],
    [0,0,0,0,0,0,1,1,1,1],
    [0,1,0,0,1,1,0,1,1,1],
    [1,1,0,0,1,1,1,0,1,1]]).T
features, labels

(array([[0, 0, 0, 0, 1],
        [0, 0, 0, 1, 1],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 1],
        [0, 1, 0, 1, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1],
        [1, 0, 1, 1, 1]]),
 array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0]))

In [217]:
tree1 = BinaryTree()
tree1.train(X=features, y=labels)

Initializing Root
Current data
 [[0 0 0 0 1]
 [0 0 0 1 1]
 [0 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 1 1]
 [0 1 0 1 1]
 [1 1 1 0 1]
 [1 1 1 1 0]
 [1 1 1 1 1]
 [1 0 1 1 1]] 
 [1 1 1 1 1 0 0 0 0 0]
Getting Best feature
Retrieved best feature :  0
Positive split : 
 [[1 1 1 0 1]
 [1 1 1 1 0]
 [1 1 1 1 1]
 [1 0 1 1 1]] 
 [0 0 0 0]
Negative split : 
 [[0 0 0 0 1]
 [0 0 0 1 1]
 [0 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 1 1]
 [0 1 0 1 1]] 
 [1 1 1 1 1 0]
Setting Child 0 of Depth 1
4
Confidence 1.0
Setting Child 1 of Depth 1
6
Confidence 0.8333333333333334
Threshold Not met 
Current data
 [[0 0 0 1]
 [0 0 1 1]
 [1 0 0 0]
 [0 0 0 0]
 [0 0 1 1]
 [1 0 1 1]] 
 [1 1 1 1 1 0]
Getting Best feature
Retrieved best feature :  0
Positive split : 
 [[1 0 0 0]
 [1 0 1 1]] 
 [1 0]
Negative split : 
 [[0 0 0 1]
 [0 0 1 1]
 [0 0 0 0]
 [0 0 1 1]] 
 [1 1 1 1]
Setting Child 0 of Depth 2
2
Confidence 0.5
Threshold Not met 
Current data
 [[0 0 0]
 [0 1 1]] 
 [1 0]
Getting Best feature
Retrieved best feature :  1
Positive split : 
 

In [286]:
labels = np.array([1,1,1,1,1,0,0,0,1,0,0,1,1,0,1,0,1])
features = np.array([
    [0,0,1,0,0,0,1,1,0,1,1,0,1,0,0,1,1],
    [0,1,0,0,0,1,1,0,1,0,1,0,1,0,1,1,1],
    [1,0,1,1,0,0,1,1,1,1,1,0,0,0,0,1,0],
    [0,1,0,0,1,1,0,1,1,1,1,0,1,0,1,1,1],
    [1,1,0,0,1,1,1,0,1,1,1,0,1,0,1,0,1]]).T
# samples = 5000
# features = 200
# labels = np.random.randint(size=(samples,), low=0, high=2)
# features = np.random.randint(size=(samples,features), low=0, high=2)
tree1 = BinaryTree(threshold=1,depth=100)
tree1.train(X=features, y=labels)

Initializing Root
Getting Best feature
Retrieved best feature :  0
Setting Child 0 of Depth 1
Confidence 0.625
Threshold Not met 
Getting Best feature
Retrieved best feature :  1
Setting Child 0 of Depth 2
Confidence 0.8333333333333334
Threshold Not met 
Getting Best feature
Retrieved best feature :  1
Setting Child 0 of Depth 3
Confidence 1.0
Threshold Met 
Setting Child 1 of Depth 3
Confidence 0.5
Threshold Not met 
Getting Best feature
Retrieved best feature :  0
Setting Child 0 of Depth 4
Confidence 1.0
Threshold Met 
Setting Child 1 of Depth 4
Confidence 1.0
Threshold Met 
Setting Child 1 of Depth 2
Confidence 1.0
Threshold Met 
Setting Child 1 of Depth 1
Confidence 0.7777777777777778
Threshold Not met 
Getting Best feature
Retrieved best feature :  1
Setting Child 0 of Depth 2
Confidence 1.0
Threshold Met 
Setting Child 1 of Depth 2
Confidence 0.6666666666666666
Threshold Not met 
Getting Best feature
Retrieved best feature :  1
Setting Child 0 of Depth 3
Confidence 0.75
Threshol

In [287]:
tree1.pretty_print()

=> Depth : 0     |||     Entropy :  0.9774178175281716   |||    Feature 0
=======> Depth : 1     |||     Entropy :  0.954434002924965   |||    Feature 1
=============> Depth : 2     |||     Entropy :  0.6500224216483541   |||    Feature 1
===================> Depth : 3     |||     Entropy :  0   |||    Confidence 1.0
===================> Depth : 3     |||     Entropy :  1.0   |||    Feature 0
=========================> Depth : 4     |||     Entropy :  0   |||    Confidence 1.0
=========================> Depth : 4     |||     Entropy :  0   |||    Confidence 1.0
=============> Depth : 2     |||     Entropy :  0   |||    Confidence 1.0
=======> Depth : 1     |||     Entropy :  0.7642045065086203   |||    Feature 1
=============> Depth : 2     |||     Entropy :  0   |||    Confidence 1.0
=============> Depth : 2     |||     Entropy :  0.9182958340544896   |||    Feature 1
===================> Depth : 3     |||     Entropy :  0.8112781244591328   |||    Feature 0
=========================>

In [267]:
tree1.predict(np.array([[1,1,1,1,0,1,1]]))